# Pesquisador financeiro

A ideia desse proejto é fazer um sistema multiagente que pesquise informações de alguma empresa
e criar um markdown que ajudará investidores terem uma noção de como está a empresa. 

### Importando libs

In [1]:
from crewai import Agent, Task, Crew, Process


### Criando tool

In [2]:
from crewai_tools import SerperDevTool

search_tool = SerperDevTool(
    n_results=1
)

/home/rmede/projects/agents-course/.venv/lib/python3.12/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [10]:
search_tool._run(query="Cayena marketplace")

{'searchParameters': {'q': 'Cayena marketplace',
  'type': 'search',
  'num': 1,
  'engine': 'google'},
 'organic': [{'title': 'Cayena - Marketplace para compra de insumos',
   'link': 'https://cayena.com/',
   'snippet': 'Facilite a rotina de compras de insumos do seu restaurante, mercado, bar ou lanchonete. Com a Cayena você abastece seu estoque em 5 minutos.',
   'position': 1,
   'sitelinks': [{'title': 'Conheça a história da Cayena',
     'link': 'https://cayena.com/a-historia-por-tras-da-cayena/'},
    {'title': 'Cayena App', 'link': 'https://cayena.com/cayena-app/'},
    {'title': 'Restaurantes', 'link': 'https://cayena.com/restaurantes/'},
    {'title': 'Cayena', 'link': 'https://loja.cayena.com/criar-conta'}]}],
 'relatedSearches': [{'query': 'cayena in english'}],
 'credits': 1}

### Criando agente pesquisador

In [4]:
# Agent pesquisador
research_role="Pesquisador financeiro sênior para a empresa: {empresa}"
research_goal = "Pesquisar a empresa, notícias e potenciais para a empresa: {empresa}"
research_backstory= """
Você é um pesquisador financeiro com o talento de encontrar informações 
    muito relevantes sobre a empresa: {empresa}.
    Você é conhecido por sua habilidade de sempre encontrar informações extremamente
    relevantes e apresentá-las de forma clara e concisa.
"""

research_agent = Agent(
    role=research_role,
    goal=research_goal,
    backstory=research_backstory,
    llm="groq/llama3-8b-8192",
    verbose=True,
    tools=[search_tool]
)



# Task pesquisador
research_description = """Conduza uma pesquisa de mercado em relação à empresa: {empresa}. Como foco em:
1. Status e saúde atual da empresa.
2. Performance histórico da empresa.
3. Maiores desafios e oportunidades.
4. Notícias e eventos recentes.
5. Perpectivas futuras e potenciais desenvolvimentos

Tenha certeza em organizar suas descobertas numa forma estruturada com seções claras.
"""

research_expected_output = """
Um documento de pesquisa muito bem organizado e estruturado, com seções cobrindo todos os requisitos pedidos
a respeito da empresa: {empresa}. Inclua fatos específicos, imagens e exemplos que forem relevantes.
"""


research_task = Task(
  description=research_description,
  expected_output=research_expected_output,
  agent=research_agent,
)

### Criando agente que irá analisar e tirar insights do conteúdo trazido pelo pesquisador

In [5]:
# Agent analista
analyst_role="Analista de mercado financeiro e escritos de relatórios com foco na empresa: {empresa}"
analyst_goal = """
Analise as informações obtidas da empresa: {empresa}
    E crie um relatório super compreensivo e muito bem estruturado 
    que apresenta insights de uma forma precisa e incrível.
"""
analyst_backstory= """
Você é um analista de mercado financeiro extremamente meticuloso e super
habilidoso, com foco na empresa: {empresa}.
Você tem o talento de para identificar padrões e extrair insights muito relevantes a partir
de dados sobre a empresa: {empresa}.
"""

analyst_agent = Agent(
    role=analyst_role,
    goal=analyst_goal,
    backstory=analyst_backstory,
    llm="groq/llama3-8b-8192",
    verbose=True
)


# Task pesquisador
analyst_description = """Analise as descobertas da pesquisa sobre a empresa: {empresa}. E crie um relatório bem estruturado.
    Você deve reportar:
    1. Comece com um resumo executivo.
    2. Inclua toda informação chave da pesquisa.
    3. Forneça análises muito boas de trends e padrões.
    4. Ofereça uma perpectiva de mercado para a empresa, especificando que essa ifnormação não deve ser usada em decisões de trade.
    5. Coloque o relatório em um formato profissiona, fácil de ler com headings claros.
"""

analyst_expected_output = """
Um relatório muito profissional e polido em relação a empresa: {empresa}
Esse relatório deve apresentar as decobertas com análises e insights incluídos.
O relatório deve ser bem estruturado com um resumo executivo, principais seções e conclusão.
"""

analyst_task = Task(
  description=analyst_description,
  expected_output=analyst_expected_output,
  agent=analyst_agent,
  context=[research_task]
)

### Criando o crew

In [6]:
crew = Crew(
    agents=[research_agent, analyst_agent],
    tasks=[research_task, analyst_task],
    process=Process.sequential
)

### Rodando o crew

In [9]:
inputs = {
    'empresa': 'Cayena marketplace'
}

results = crew.kickoff(inputs=inputs)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Pesquisador financeiro sênior para a empresa: Cayena marketplace                                        │
│                                                                                                                 │
│  Task: Conduza uma pesquisa de mercado em relação à empresa: Cayena marketplace. Como foco em:                  │
│  1. Status e saúde atual da empresa.                                                                            │
│  2. Performance histórico da empresa.                                                                           │
│  3. Maiores desafios e oportunidades.                                                                           │
│  4. Notícias e eventos recentes.                                                                                │
│  5. Perpectivas futuras e potenciais desenvolvimentos                                                           │
│                                                                                                                 │
│  Tenha certeza em organizar suas descobertas numa forma estruturada com seções claras.                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1 validation error for SerperDevToolSchema
search_query
  Input should be a valid string [type=string_type, input_value={'description': 'Cayena m...esearch', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type.
 Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'



╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Pesquisador financeiro sênior para a empresa: Cayena marketplace                                        │
│                                                                                                                 │
│  Thought: Thought: Vou precisar realizar uma pesquisa extensa para coletar informações detalhadas sobre a       │
│  empresa Cayena marketplace. Para isso, vou utilizar o ferramenta de busca do Serper para coletar informações   │
│  relevantes.                                                                                                    │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": {\"description\": \"Cayena marketplace market research\", \"type\": \"str\"}}"             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│  I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 1        │
│  validation error for SerperDevToolSchema                                                                       │
│  search_query                                                                                                   │
│    Input should be a valid string [type=string_type, input_value={'description': 'Cayena m...esearch', 'type':  │
│  'str'}, input_type=dict]                                                                                       │
│      For further information visit https://errors.pydantic.dev/2.11/v/string_type.                              │
│   Tool Search the internet with Serper accepts these inputs: Tool Name: Search the internet with Serper         │
│  Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the         │
│  internet', 'type': 'str'}}                                                                                     │
│  Tool Description: A tool that can be used to search the internet with a search_query. Supports different       │
│  search types: 'search' (default), 'news'.                                                                      │
│  Moving on then. I MUST either use a tool (use one at time) OR give my best final answer not both at the same   │
│  time. When responding, I must use the following format:                                                        │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: you should always think about what to do                                                              │
│  Action: the action to take, should be one of [Search the internet with Serper]                                 │
│  Action Input: the input to the action, dictionary enclosed in curly braces                                     │
│  Observation: the result of the action                                                                          │
│  ```                                                                                                            │
│  This Thought/Action/Action Input/Result can repeat N times. Once I know the final answer, I must return the    │
│  following format:                                                                                              │
│                                                                                                                 │
│  ```                                                                                                            │
│  Thought: I now can give a great answer                                                                         │
│  Final Answer: Your final answer must be the great and the most complete as possible, it must be outcome        │
│  described                                                                                                      │
│                                                                                                                 │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 Received None or empty response from LLM call.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.
 An unknown error occurred. Please check the details below.
 Error details: Invalid response from LLM call - None or empty.


ValueError: Invalid response from LLM call - None or empty.

In [8]:
print(results.raw)

NameError: name 'results' is not defined